In [291]:
import pandas as pd
pd.options.display.max_columns =200
import numpy as np
from datetime import datetime, timedelta

#### Inputs

p = County Population  
cl = County 14 day caseload  

#### Parameters
base = contact tracers per total population   
con = contacts per case  
case_int = number of confirmed case interviews per day  
cont_not = number of contact notifications per day  
cont_follow_up = number of contact follow ups per day  
follow_freq = follow up frequency (per contact per week)  

fl = total number of follow up case encounters  


Demand = 
base_ct_need = (p/10000)*base
case_int_need = (cl/case_int)/5
contact_notify_need = ((cl*con)/cont_not)/5

contact_follow_up = sum(dailycontact follow up)


daily contact follow up: (day number of cases * (14+(-days from current)) / 7)* follow_freq * contacts per case

final_need = base_ct_need + case_int_need + contact_notify_need + contact_follow_up


In [292]:
#### replace with desired (or most recent) date in NY Times dataset:
covid_data_update_date = '2020-06-08'

#### Parameters for GW model
base_need = 15 
contacts = 10
case_interviews = 6
cont_notifications = 12
contact_followup = 32
followup_freq = 7

In [293]:
## set up 14 day period
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format=True)
N = 14
date_N_days_ago = data_date_dt - timedelta(days=N)

date_N1_days_ago = data_date_dt - timedelta(days=N+1)

In [294]:
## data imports
svi = pd.read_csv("data/svi/svi_2018_counties_state_ranked.csv")
covid = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv")

In [295]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)

In [296]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt
 

In [297]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases']=covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
covid_last15.sort_index(inplace=True)


In [298]:
## select just last 14 days now that we have daily differences with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt']>date_N_days_ago) & (covid_last15['dt']<= data_date_dt)].copy()


In [299]:
## calculate follow up encounter demand
covid_last14['fe_demand'] = covid_last14['new_cases']*(14+((covid_last14['dt']-data_date_dt).dt.days))/7*follow_freq*con


In [300]:
## group by to get case load and follow up demand values for each county

covid_last14_stats = covid_last14.groupby(['fips'])['new_cases','fe_demand'].sum()

<ipython-input-300-27d2457dc4a4>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  covid_last14_stats = covid_last14.groupby(['fips'])['new_cases','fe_demand'].sum()


In [301]:
# adding population information from CDC svi dataset

covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)

covid_last14_stats = covid_last14_stats.merge(svi[['FIPS','E_TOTPOP']],left_on='fips',right_on='FIPS')

In [302]:
## calculations for each type of demand

covid_last14_stats['base_ct'] = (covid_last14_stats['E_TOTPOP']/100000)*base_need
covid_last14_stats['case_int_need'] = covid_last14_stats['new_cases']/case_interviews/5
covid_last14_stats['contact_notify_need'] = (covid_last14_stats['new_cases']*contacts/cont_notifications)/5
covid_last14_stats['contact_follow_need'] = covid_last14_stats['fe_demand']/(contact_followup*5*2)
covid_last14_stats['covid_need'] = covid_last14_stats['case_int_need']+covid_last14_stats['contact_notify_need']+covid_last14_stats['contact_follow_need']
covid_last14_stats['total_need'] = covid_last14_stats[['covid_need','base_ct']].max(axis=1)

In [303]:
## saving simplified form
covid_demand = covid_last14_stats[['fips','total_need']]

In [304]:
covid_demand

,fips,total_need
0,1001,42.50000
1,1003,31.21605
2,1005,33.78125
3,1007,7.97500
4,1009,10.95000
...,...,...
2981,56037,6.61755
2982,56039,3.45885
2983,56041,7.41250
2984,56043,2.81875
